<a href="https://colab.research.google.com/github/nickklos10/SerieA_Machine_Learning_Predictions_2025/blob/main/SerieA_Team_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

url_market = "https://www.transfermarkt.us/serie-a/startseite/wettbewerb/IT1/saison_id/2004"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36'
}

response = requests.get(url_market, headers=headers)

if response.status_code == 200:
    print("Request successful!")
    html_content = response.text
else:
    print(f"Request failed with status code {response.status_code}")

Request successful!


In [ ]:
soup = BeautifulSoup(html_content, 'html.parser')

season_select = soup.find('select', {'name': 'saison_id'})
year = season_select.find('option', selected=True).get('value')


table = soup.find('table', class_='items')

In [ ]:
teams = []
squad_sizes = []
average_ages = []
foreigners = []
average_market_values = []
total_market_values = []
years = []

In [ ]:
for row in table.find_all('tr', class_=['odd', 'even']):

    team = row.find('a', title=True).get('title')
    teams.append(team)

    squad_size = row.find_all('td', class_='zentriert')[1].text
    squad_sizes.append(squad_size)

    avg_age = row.find_all('td', class_='zentriert')[2].text
    average_ages.append(avg_age)

    foreigner_count = row.find_all('td', class_='zentriert')[3].text
    foreigners.append(foreigner_count)

    avg_market_value = row.find_all('td', class_='rechts')[0].text
    average_market_values.append(avg_market_value)

    total_market_value = row.find_all('td', class_='rechts')[1].text.strip('€')
    total_market_values.append(total_market_value)


    years.append(year)

In [ ]:
df = pd.DataFrame({
    'Team': teams,
    'Squad Size': squad_sizes,
    'Average Age': average_ages,
    'Foreigners': foreigners,
    'Average Market Value': average_market_values,
    'Total Market Value (€)': total_market_values,
    'Year': years  # Add the year column
})

csv_file = 'serie_a_market_values.csv'

In [ ]:
print(df)

                 Team Squad Size Average Age Foreigners Average Market Value  \
0         Juventus FC         39        26.2         14               €7.82m   
1            AC Milan         37        27.2         17               €7.40m   
2         Inter Milan         50        24.9         25               €4.81m   
3             AS Roma         40        23.6         10               €5.04m   
4      ACF Fiorentina         48        25.5         13               €2.33m   
5            Parma FC         44        23.8          8               €2.50m   
6      Udinese Calcio         34        25.6         17               €2.47m   
7          US Palermo         29        25.5          5               €2.72m   
8            SS Lazio         45        26.3         12               €1.71m   
9            AC Siena         40        27.6          9               €1.37m   
10       UC Sampdoria         37        25.8          3               €1.38m   
11      Chievo Verona         42        

In [ ]:
def parse_fee(value):

    if isinstance(value, (float, int)):
        return value

    value = value.replace('€', '').replace('£', '').replace('$', '').strip()

    if 'Loan fee:' in value or 'Loan Fee:' in value:
        fee_value = value.split(':')[1]
        return parse_numeric_value(fee_value)

    elif 'm' in value or 'k' in value:
        return parse_numeric_value(value)


    elif value.lower() in ['free transfer', 'loan', 'loan transfer', 'end of loan']:
        return 0.0
    else:
        return 'Other'

def parse_numeric_value(value):
    if isinstance(value, (float, int)):
        return value

    if isinstance(value, str):
        value = value.replace('€', '').replace('$', '').replace('£', '').replace(',', '.').strip()

        if 'm' in value.lower():
            return float(value.lower().replace('m', '')) * 1e6
        elif 'k' in value.lower():
            return float(value.lower().replace('k', '')) * 1e3
        else:
            try:
                return float(value)
            except ValueError:
                return 'Other'
    else:
        return 'Other'

In [ ]:
df['Total Market Value (€)'] = df['Total Market Value (€)'].apply(parse_fee)
df['Average Market Value'] = df['Average Market Value'].apply(parse_numeric_value)

if os.path.isfile(csv_file):
    df.to_csv(csv_file, mode='a', index=False, header=False)
else:
    df.to_csv(csv_file, mode='w', index=False, header=True)

print("Data cleaned and saved to csv file")

Data cleaned and saved to csv file
